# ML-Based Automatic Mask Generation for Pyramid Blending
## With Interactive File Upload

This notebook uses MediaPipe for automatic face detection and mask generation,
combined with the pyramid blending algorithm from Proj_opt1.ipynb.

**Upload your own images directly in the notebook!**

In [ ]:
# Install required packages
import sys
import subprocess

def install_package(package):
    try:
        # For mediapipe, check if we need Colab-compatible version
        if package.startswith('mediapipe'):
            import mediapipe as mp
            print(f"✅ {package} already installed (v{mp.__version__})")
        else:
            __import__(package.split('[')[0])
            print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✅ {package} installed")

# For Google Colab, use mediapipe>=0.10.14
# For local, any version works
try:
    from google.colab import files
    # Running on Colab - install specific version
    print("🔍 Running on Google Colab - installing Colab-compatible versions...\n")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "mediapipe>=0.10.14"])
    print("✅ MediaPipe (Colab version) installed")
except ImportError:
    # Not on Colab - install normally
    install_package("mediapipe")

install_package("opencv-python")
install_package("numpy")
install_package("matplotlib")
install_package("pillow")
install_package("ipywidgets")

print("\n✅ All packages ready!")

In [ ]:
# Import libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import mediapipe as mp
import time
import io

# Try to import Google Colab files module
try:
    from google.colab import files as colab_files
    COLAB_AVAILABLE = True
    print("✅ Running on Google Colab")
except ImportError:
    COLAB_AVAILABLE = False
    print("✅ Running on local Jupyter")

print(f"✅ NumPy {np.__version__}")
print(f"✅ OpenCV {cv2.__version__}")
print(f"✅ MediaPipe {mp.__version__}")

## Core Functions from Proj_opt1.ipynb

In [ ]:
def pad_image(image, kernel, pad_type):
    """Pad image for convolution (from Proj_opt1.ipynb)."""
    height, width = kernel.shape
    pad_height = height // 2
    pad_width = width // 2
    
    if pad_type == 'clip':
        padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='constant')
    elif pad_type == 'wrap':
        padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='wrap')
    elif pad_type == 'copy_edge':
        padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='edge')
    elif pad_type == 'reflect':
        padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='reflect')
    else:
        raise ValueError("Invalid padding type.")
    
    return padded_image

def conv2(image, kernel, pad_type):
    """Perform 2D convolution using matrix slicing (from Proj_opt1.ipynb)."""
    assert len(kernel.shape) == 2, "Kernel must be 2D."
    
    if len(image.shape) == 2:
        image = image[:, :, np.newaxis]
    
    height, width, channels = image.shape
    k_height, k_width = kernel.shape
    
    padded_image = pad_image(image, kernel, pad_type)
    output = np.zeros((height, width, channels))
    
    flipped_kernel = kernel[::-1, ::-1]
    
    for c in range(channels):
        windows = np.lib.stride_tricks.sliding_window_view(
            padded_image[:, :, c], (k_height, k_width)
        )
        output[:, :, c] = np.tensordot(windows, flipped_kernel, axes=((2, 3), (0, 1)))
    
    if output.shape[2] == 1:
        output = output[:, :, 0]
    
    return output

def downsample(image):
    """Downsample by factor of 2."""
    return image[::2, ::2]

def upsample(image, target_size):
    """Upsample with smoothing."""
    if len(image.shape) == 3:
        upsampled = np.zeros((target_size[0], target_size[1], image.shape[2]), dtype=image.dtype)
        upsampled[::2, ::2, :] = image
    else:
        upsampled = np.zeros(target_size, dtype=image.dtype)
        upsampled[::2, ::2] = image
    
    upsampled = cv2.GaussianBlur(upsampled, (5, 5), 0)
    return upsampled

def ComputePyr(input_image, num_layers):
    """Compute Gaussian and Laplacian pyramids (from Proj_opt1.ipynb)."""
    max_layers = int(np.floor(np.log2(min(input_image.shape[:2])))) - 1
    num_layers = min(num_layers, max_layers)
    
    kernel = cv2.getGaussianKernel(5, 1)
    kernel = np.outer(kernel, kernel)
    
    gPyr = [input_image.astype(np.float32)]
    lPyr = []
    
    for i in range(num_layers):
        smoothed = conv2(gPyr[-1], kernel, 'wrap')
        downsampled = downsample(smoothed)
        gPyr.append(downsampled)
        
        target_size = (gPyr[-2].shape[0], gPyr[-2].shape[1])
        upsampled = upsample(downsampled, target_size)
        laplacian = gPyr[-2] - upsampled
        lPyr.append(laplacian)
    
    lPyr.append(gPyr[-1])
    return gPyr, lPyr

def create_smooth_edge_mask(mask, edge_width=80, num_passes=3):
    """
    Create ultra-smooth edge transitions using multi-pass gaussian blur.
    
    Args:
        mask: Binary mask (0-255)
        edge_width: Width of the smooth edge region in pixels
        num_passes: Number of gaussian blur passes for smoother edges
    
    Returns:
        Smoothed mask with gradual falloff at edges
    """
    mask_float = mask.astype(float) / 255.0
    
    # Multi-pass gaussian blur for extremely smooth edges
    for i in range(num_passes):
        # Progressively larger kernels
        kernel_size = edge_width // (num_passes - i) if i < num_passes - 1 else edge_width
        kernel_size = kernel_size if kernel_size % 2 == 1 else kernel_size + 1
        mask_float = cv2.GaussianBlur(mask_float, (kernel_size, kernel_size), 0)
    
    # Apply distance transform for natural falloff
    mask_binary = (mask > 127).astype(np.uint8)
    dist_transform = cv2.distanceTransform(mask_binary, cv2.DIST_L2, 5)
    
    # Normalize distance transform
    if dist_transform.max() > 0:
        dist_transform = dist_transform / dist_transform.max()
    
    # Combine gaussian blur with distance transform for optimal smoothness
    # Use sigmoidal function for smooth transition
    dist_weight = np.clip(dist_transform * 2, 0, 1)
    smooth_mask = mask_float * 0.7 + dist_weight * 0.3
    
    return np.clip(smooth_mask, 0, 1)

def match_color_histogram(source, target, mask):
    """
    Advanced color matching using histogram matching in LAB color space.
    
    Args:
        source: Source image (RGB)
        target: Target image (RGB)
        mask: Mask indicating region to match (0-1 float)
    
    Returns:
        Color-matched source image
    """
    # Convert to LAB color space for better color matching
    source_lab = cv2.cvtColor(source.astype(np.uint8), cv2.COLOR_RGB2LAB).astype(float)
    target_lab = cv2.cvtColor(target.astype(np.uint8), cv2.COLOR_RGB2LAB).astype(float)
    
    # Match mean and std for each channel in masked region
    mask_3d = mask[..., None] if len(mask.shape) == 2 else mask
    
    matched_lab = source_lab.copy()
    for c in range(3):
        source_channel = source_lab[:, :, c]
        target_channel = target_lab[:, :, c]
        
        # Calculate statistics in masked region
        mask_area = np.sum(mask_3d > 0.1)
        if mask_area > 0:
            source_mean = np.sum(source_channel * mask_3d[:, :, 0]) / mask_area
            source_std = np.sqrt(np.sum((source_channel - source_mean)**2 * mask_3d[:, :, 0]) / mask_area)
            
            target_mean = np.sum(target_channel * mask_3d[:, :, 0]) / mask_area
            target_std = np.sqrt(np.sum((target_channel - target_mean)**2 * mask_3d[:, :, 0]) / mask_area)
            
            # Match histogram
            if source_std > 1e-5:
                matched_lab[:, :, c] = (source_channel - source_mean) * (target_std / source_std) + target_mean
    
    # Convert back to RGB
    matched_lab = np.clip(matched_lab, 0, 255).astype(np.uint8)
    matched_rgb = cv2.cvtColor(matched_lab, cv2.COLOR_LAB2RGB)
    
    return matched_rgb

def blend_with_pyramids(target_image, clipped_image, mask, levels=6):
    """
    OPTIMIZED: Blend using Laplacian pyramids with enhanced edge smoothness.
    
    Improvements:
    - Multi-pass gaussian blur for ultra-smooth mask edges
    - Advanced color histogram matching in LAB space
    - Progressive mask feathering across pyramid levels
    - Edge-aware blending to preserve facial features
    """
    clipped_image = cv2.resize(clipped_image, (target_image.shape[1], target_image.shape[0]))
    mask = cv2.resize(mask, (target_image.shape[1], target_image.shape[0]))
    
    # Step 1: Create ultra-smooth edge mask
    print("   Preprocessing mask for smooth edges...")
    mask_normalized = mask / 255.0
    smoothed_mask = create_smooth_edge_mask(mask, edge_width=100, num_passes=4)
    
    # Step 2: Advanced color matching in LAB space
    print("   Performing advanced color matching...")
    clipped_adjusted = match_color_histogram(clipped_image, target_image, smoothed_mask)
    
    # Step 3: Build pyramids
    print("   Building image pyramids...")
    gPyr_target, lPyr_target = ComputePyr(target_image, levels)
    gPyr_clipped, lPyr_clipped = ComputePyr(clipped_adjusted, levels)
    gPyr_mask, _ = ComputePyr(smoothed_mask, levels)
    
    # Step 4: Multi-scale blending with progressive feathering
    print("   Blending across pyramid levels...")
    lPyr_blended = []
    for level, (l_target, l_clipped, g_mask) in enumerate(zip(lPyr_target, lPyr_clipped, gPyr_mask)):
        # Apply additional smoothing to mask at each level for better transitions
        level_mask = g_mask
        if level < len(gPyr_mask) - 1:
            kernel_size = 15 - level * 2
            kernel_size = max(3, kernel_size if kernel_size % 2 == 1 else kernel_size + 1)
            level_mask = cv2.GaussianBlur(g_mask, (kernel_size, kernel_size), 0)
        
        # Blend with progressive mask
        if len(level_mask.shape) == 2:
            level_mask = level_mask[..., None]
        
        blended = l_target * (1 - level_mask) + l_clipped * level_mask
        lPyr_blended.append(blended)
    
    # Step 5: Reconstruct from pyramid
    print("   Reconstructing final image...")
    blended_image = lPyr_blended[-1]
    for i in range(len(lPyr_blended) - 2, -1, -1):
        blended_image = upsample(blended_image, lPyr_blended[i].shape[:2])
        blended_image = cv2.add(blended_image, lPyr_blended[i])
    
    # Step 6: Final blend with ultra-smooth mask
    smoothed_mask_3d = smoothed_mask[..., None] if len(smoothed_mask.shape) == 2 else smoothed_mask
    final_result = target_image * (1 - smoothed_mask_3d) + blended_image * smoothed_mask_3d
    
    # Step 7: Apply subtle post-processing for seamless edges
    print("   Applying edge refinement...")
    final_result = np.clip(final_result, 0, 255).astype(np.uint8)
    
    # Bilateral filter only in edge region for edge-aware smoothing
    edge_region = (smoothed_mask > 0.1) & (smoothed_mask < 0.9)
    if edge_region.any():
        filtered = cv2.bilateralFilter(final_result, d=9, sigmaColor=75, sigmaSpace=75)
        edge_blend = edge_region.astype(float) * (1 - np.abs(smoothed_mask - 0.5) * 2)
        edge_blend = edge_blend[..., None] if len(edge_blend.shape) == 2 else edge_blend
        final_result = (final_result * (1 - edge_blend) + filtered * edge_blend).astype(np.uint8)
    
    return final_result

print("✅ OPTIMIZED pyramid blending functions loaded")

## AI-Powered Face Detection and Mask Generation

In [ ]:
class FaceDetector:
    """AI-powered face detection using MediaPipe."""
    
    # Face oval landmark indices for mask generation
    FACE_OVAL_INDICES = [
        10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
        397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
        172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109
    ]
    
    # Key facial landmarks
    LEFT_EYE_CENTER = 33
    RIGHT_EYE_CENTER = 263
    NOSE_TIP = 1
    MOUTH_CENTER = 13
    
    def __init__(self):
        """Initialize MediaPipe Face Mesh - supports both old and new API."""
        import mediapipe as mp
        
        # Try new API first (mediapipe >= 0.10.8)
        try:
            from mediapipe.tasks import python
            from mediapipe.tasks.python import vision
            import os
            import urllib.request
            
            # Download model if not exists
            model_path = 'face_landmarker.task'
            if not os.path.exists(model_path):
                print("   Downloading face landmarker model...")
                model_url = 'https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task'
                urllib.request.urlretrieve(model_url, model_path)
                print("   Model downloaded!")
            
            base_options = python.BaseOptions(model_asset_path=model_path)
            options = vision.FaceLandmarkerOptions(
                base_options=base_options,
                running_mode=vision.RunningMode.IMAGE,
                num_faces=1,
                min_face_detection_confidence=0.5,
                min_face_presence_confidence=0.5,
                min_tracking_confidence=0.5
            )
            self.face_mesh = vision.FaceLandmarker.create_from_options(options)
            self.use_new_api = True
            print("   Using new MediaPipe API (tasks.vision)")
            
        except Exception as e:
            # Fallback to old API (mediapipe < 0.10.8)
            try:
                self.face_mesh = mp.solutions.face_mesh.FaceMesh(
                    static_image_mode=True,
                    max_num_faces=1,
                    refine_landmarks=True,
                    min_detection_confidence=0.5
                )
                self.use_new_api = False
                print("   Using legacy MediaPipe API (solutions)")
            except AttributeError:
                raise ImportError(
                    f"Could not initialize MediaPipe Face Mesh. Error: {e}"
                )
    
    def detect_face(self, image):
        """Detect face and extract 468 landmarks."""
        h, w = image.shape[:2]
        
        if self.use_new_api:
            # New API
            import mediapipe as mp
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
            results = self.face_mesh.detect(mp_image)
            
            if not results.face_landmarks:
                return None
            
            landmarks = results.face_landmarks[0]
            landmark_points = []
            for lm in landmarks:
                x = int(lm.x * w)
                y = int(lm.y * h)
                landmark_points.append((x, y))
        else:
            # Old API
            results = self.face_mesh.process(image)
            
            if not results.multi_face_landmarks:
                return None
            
            landmarks = results.multi_face_landmarks[0]
            landmark_points = []
            for lm in landmarks.landmark:
                x = int(lm.x * w)
                y = int(lm.y * h)
                landmark_points.append((x, y))
        
        key_points = {
            'left_eye': np.array(landmark_points[self.LEFT_EYE_CENTER]),
            'right_eye': np.array(landmark_points[self.RIGHT_EYE_CENTER]),
            'nose': np.array(landmark_points[self.NOSE_TIP]),
            'mouth': np.array(landmark_points[self.MOUTH_CENTER])
        }
        
        return {
            'landmarks': landmark_points,
            'key_points': key_points
        }
    
    def create_face_mask(self, image, face_data, expand_ratio=0.18):
        """Create mask from face oval landmarks."""
        h, w = image.shape[:2]
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Get face oval points
        oval_points = [face_data['landmarks'][i] for i in self.FACE_OVAL_INDICES]
        oval_points = np.array(oval_points, dtype=np.int32)
        
        # Expand contour for better coverage
        center = np.mean(oval_points, axis=0)
        expanded_points = center + (oval_points - center) * (1 + expand_ratio)
        expanded_points = expanded_points.astype(np.int32)
        
        # Fill mask
        cv2.fillPoly(mask, [expanded_points], 255)
        
        # Apply light smoothing
        mask_float = mask.astype(float) / 255.0
        mask_float = cv2.GaussianBlur(mask_float, (15, 15), 0)
        mask_float = np.clip(mask_float, 0, 1)
        
        return (mask_float * 255).astype(np.uint8)

print("✅ FaceDetector class loaded")

In [ ]:
class FaceAligner:
    """Align faces using feature-based transformation."""
    
    def compute_alignment_transform(self, source_key_points, target_key_points):
        """Compute scale and rotation from eye positions."""
        src_left = source_key_points['left_eye']
        src_right = source_key_points['right_eye']
        tgt_left = target_key_points['left_eye']
        tgt_right = target_key_points['right_eye']
        
        # Calculate scale from eye distance
        src_eye_dist = np.linalg.norm(src_right - src_left)
        tgt_eye_dist = np.linalg.norm(tgt_right - tgt_left)
        scale = tgt_eye_dist / src_eye_dist
        
        # Calculate rotation from eye angle
        src_angle = np.arctan2(src_right[1] - src_left[1], src_right[0] - src_left[0])
        tgt_angle = np.arctan2(tgt_right[1] - tgt_left[1], tgt_right[0] - tgt_left[0])
        rotation = tgt_angle - src_angle
        
        # Compute centers
        src_center = (src_left + src_right) / 2
        tgt_center = (tgt_left + tgt_right) / 2
        
        return {
            'scale': scale,
            'rotation': rotation,
            'src_center': src_center,
            'tgt_center': tgt_center
        }
    
    def apply_alignment(self, image, transform):
        """Apply scale and rotation transformation."""
        center = (float(transform['src_center'][0]), float(transform['src_center'][1]))
        angle = float(np.degrees(transform['rotation']))
        scale = float(transform['scale'])
        
        M = cv2.getRotationMatrix2D(center, angle, scale)
        aligned = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
        
        return aligned
    
    def place_on_canvas(self, aligned_image, target_shape, tgt_center, src_center):
        """Place aligned image on canvas matching target size."""
        canvas = np.zeros(target_shape, dtype=aligned_image.dtype)
        
        offset = (tgt_center - src_center).astype(int)
        
        y_start = max(0, offset[1])
        y_end = min(canvas.shape[0], offset[1] + aligned_image.shape[0])
        x_start = max(0, offset[0])
        x_end = min(canvas.shape[1], offset[0] + aligned_image.shape[1])
        
        src_y_start = max(0, -offset[1])
        src_y_end = src_y_start + (y_end - y_start)
        src_x_start = max(0, -offset[0])
        src_x_end = src_x_start + (x_end - x_start)
        
        canvas[y_start:y_end, x_start:x_end] = aligned_image[src_y_start:src_y_end, src_x_start:src_x_end]
        
        return canvas

print("✅ FaceAligner class loaded")

## Main Face Swap Function

In [ ]:
def auto_face_swap(source_image, target_image, visualize=True, pyramid_levels=6):
    """
    Automatic face swapping with AI detection.
    
    Args:
        source_image: RGB image array (face to swap IN)
        target_image: RGB image array (face to swap ONTO)
        visualize: Whether to display results
        pyramid_levels: Number of pyramid levels for blending
    
    Returns:
        Dictionary with results
    """
    print("\n" + "="*70)
    print("🤖 AI-Powered Face Swap")
    print("="*70 + "\n")
    
    print(f"📊 Image sizes:")
    print(f"   Source: {source_image.shape}")
    print(f"   Target: {target_image.shape}")
    
    # Step 1: Detect faces
    print("\n🔍 Step 1: Detecting faces...")
    detector = FaceDetector()
    
    source_face = detector.detect_face(source_image)
    if source_face is None:
        print("❌ ERROR: No face detected in source image!")
        return None
    
    target_face = detector.detect_face(target_image)
    if target_face is None:
        print("❌ ERROR: No face detected in target image!")
        return None
    
    print(f"   ✅ Source: {len(source_face['landmarks'])} landmarks")
    print(f"   ✅ Target: {len(target_face['landmarks'])} landmarks")
    
    # Step 2: Align faces
    print("\n📐 Step 2: Aligning facial features...")
    aligner = FaceAligner()
    
    transform = aligner.compute_alignment_transform(
        source_face['key_points'],
        target_face['key_points']
    )
    
    print(f"   Scale factor: {transform['scale']:.2f}")
    print(f"   Rotation: {np.degrees(transform['rotation']):.1f}°")
    
    aligned_source = aligner.apply_alignment(source_image, transform)
    source_canvas = aligner.place_on_canvas(
        aligned_source,
        target_image.shape,
        transform['tgt_center'],
        transform['src_center']
    )
    
    # Step 3: Generate mask
    print("\n🎭 Step 3: Generating automatic face mask...")
    canvas_face = detector.detect_face(source_canvas)
    if canvas_face:
        mask = detector.create_face_mask(source_canvas, canvas_face, expand_ratio=0.15)
        print("   ✅ Mask generated from aligned face contours")
    else:
        mask = detector.create_face_mask(target_image, target_face, expand_ratio=0.15)
        print("   ⚠️  Using target face for mask (fallback)")
    
    # Step 4: Pyramid blending
    print(f"\n🎨 Step 4: Performing pyramid blending ({pyramid_levels} levels)...")
    start_time = time.time()
    
    blended = blend_with_pyramids(target_image, source_canvas, mask, levels=pyramid_levels)
    
    elapsed = (time.time() - start_time) * 1000
    print(f"   ✅ Blending complete in {elapsed:.1f}ms")
    
    # Visualization
    if visualize:
        print("\n📊 Step 5: Creating visualization...")
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        
        axes[0, 0].imshow(source_image)
        axes[0, 0].set_title('Source (face to swap IN)', fontsize=12, fontweight='bold')
        axes[0, 0].axis('off')
        
        axes[0, 1].imshow(target_image)
        axes[0, 1].set_title('Target (face to swap ONTO)', fontsize=12, fontweight='bold')
        axes[0, 1].axis('off')
        
        axes[0, 2].imshow(source_canvas)
        axes[0, 2].set_title('Aligned Source on Canvas', fontsize=12, fontweight='bold')
        axes[0, 2].axis('off')
        
        axes[1, 0].imshow(mask, cmap='gray')
        axes[1, 0].set_title('Auto-Generated Mask', fontsize=12, fontweight='bold')
        axes[1, 0].axis('off')
        
        axes[1, 1].imshow(blended)
        axes[1, 1].set_title('✨ FINAL RESULT ✨', fontsize=12, fontweight='bold', color='green')
        axes[1, 1].axis('off')
        
        # Side-by-side comparison
        comparison = np.hstack([target_image, blended])
        axes[1, 2].imshow(comparison)
        axes[1, 2].set_title('Before | After', fontsize=12, fontweight='bold')
        axes[1, 2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    print("\n" + "="*70)
    print("✅ Face swap completed successfully!")
    print("="*70 + "\n")
    
    return {
        'result': blended,
        'source_canvas': source_canvas,
        'mask': mask,
        'scale': transform['scale'],
        'rotation_degrees': np.degrees(transform['rotation']),
        'blending_time_ms': elapsed
    }

print("✅ Main pipeline function loaded")

## Image Upload Helper Function

In [ ]:
def upload_image(prompt="Upload an image"):
    """
    Upload image - works in both Google Colab and local Jupyter.
    Returns: (image_array, filename)
    """
    if COLAB_AVAILABLE:
        # Google Colab upload
        print(f"📤 {prompt}")
        print("   Click 'Choose Files' button below...\n")
        
        uploaded = colab_files.upload()
        
        if not uploaded:
            print("❌ No file uploaded!")
            return None, None
        
        filename = list(uploaded.keys())[0]
        image_bytes = uploaded[filename]
        image = np.array(Image.open(io.BytesIO(image_bytes)))
        
    else:
        # Local Jupyter upload using ipywidgets
        from ipywidgets import FileUpload
        from IPython.display import display, clear_output
        
        print(f"📤 {prompt}")
        print("   Click 'Upload' button below to select a file...\n")
        
        uploader = FileUpload(accept='image/*', multiple=False)
        display(uploader)
        
        # Wait for upload
        import time
        while len(uploader.value) == 0:
            time.sleep(0.1)
        
        clear_output()
        
        filename = list(uploader.value.keys())[0]
        image_bytes = uploader.value[filename]['content']
        image = np.array(Image.open(io.BytesIO(image_bytes)))
    
    # Convert to RGB if needed
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    
    print(f"✅ Loaded: {filename}")
    print(f"   Shape: {image.shape}\n")
    
    return image, filename

print("✅ Image upload helper function loaded")

## Step 1: Upload Source Image

Run the cell below to upload the **source image** (face to swap IN).

In [ ]:
# Upload source image
source_image, source_filename = upload_image("Upload SOURCE image (face to swap IN)")

if source_image is not None:
    # Display preview
    plt.figure(figsize=(6, 6))
    plt.imshow(source_image)
    plt.title(f'Source: {source_filename}', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## Step 2: Upload Target Image

Run the cell below to upload the **target image** (face to swap ONTO).

In [ ]:
# Upload target image
target_image, target_filename = upload_image("Upload TARGET image (face to swap ONTO)")

if target_image is not None:
    # Display preview
    plt.figure(figsize=(6, 6))
    plt.imshow(target_image)
    plt.title(f'Target: {target_filename}', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## Step 3: Preview Both Images

In [ ]:
# Display both images side by side
if 'source_image' in locals() and source_image is not None and 'target_image' in locals() and target_image is not None:
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(source_image)
    axes[0].set_title(f'Source: {source_filename}', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(target_image)
    axes[1].set_title(f'Target: {target_filename}', fontsize=12, fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Both images loaded successfully!")
    print("   Ready to perform face swap.")
else:
    print("❌ Please upload both source and target images first!")

## Step 4: Run Face Swap

In [ ]:
# Run the face swap
if 'source_image' in locals() and source_image is not None and 'target_image' in locals() and target_image is not None:
    result = auto_face_swap(
        source_image=source_image,
        target_image=target_image,
        visualize=True,
        pyramid_levels=6
    )
    
    if result:
        print("\n📊 Results Summary:")
        print(f"   Scale: {result['scale']:.2f}x")
        print(f"   Rotation: {result['rotation_degrees']:.1f}°")
        print(f"   Blending time: {result['blending_time_ms']:.1f}ms")
        
        # Save results
        print("\n💾 Saving results...")
        cv2.imwrite('ml_face_swap_result.png', cv2.cvtColor(result['result'], cv2.COLOR_RGB2BGR))
        cv2.imwrite('ml_face_swap_mask.png', result['mask'])
        cv2.imwrite('ml_face_swap_canvas.png', cv2.cvtColor(result['source_canvas'], cv2.COLOR_RGB2BGR))
        print("✅ Results saved:")
        print("   - ml_face_swap_result.png")
        print("   - ml_face_swap_mask.png")
        print("   - ml_face_swap_canvas.png")
else:
    print("❌ Please upload both source and target images first!")

## Done!

### Features:
- **Interactive File Upload** - Works in both Google Colab and local Jupyter
- **MediaPipe AI Detection** - 468 facial landmarks
- **Automatic Alignment** - Feature-based scale and rotation
- **Auto Mask Generation** - From face contours
- **Pyramid Blending** - Seamless 6-level blending

### To try again:
Simply re-run the upload cells (Step 1 & 2) to select different images!